<a href="https://colab.research.google.com/github/joshuavachon25/py-recipe/blob/main/py_recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialisation des librairies


In [ ]:
pip install beautifulsoup4

In [21]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import json
import re

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Lister les URL des livres

In [ ]:
# Initialisation des variables
baseURL = "https://archive.org/details/cbk?and%5B%5D=languageSorter%3A%22English%22&and%5B%5D=mediatype%3A%22texts%22&sort=date&page="
deb = 1
fin = 40
livres = []

# Deux boucles for; La première parcourt les pages, la seconde les liens sur chaque page.
for page in range(deb,fin,1):
    html = urlopen(baseURL + str(page))
    bs = BeautifulSoup(html.read(), 'html.parser')
    
    for lien in bs.find('div',{'class':'results'}).find_all('div', {'class':'item-ttl'}):
        livres.append('https://archive.org' + lien.a.attrs['href'])

In [ ]:
metadata = []
g = open('metadata.csv', 'w', encoding='UTF8')
writer = csv.writer(g)
writer.writerow(['titre', 'date', 'editeur', 'fichier'])
for livre in livres:
    html = urlopen(livre)
    bs = BeautifulSoup(html.read(), 'html.parser')

    try:
        titre = bs.find('h1', {'class': 'item-title'}).span.text
    except:
        titre = ""

    try:
        date = bs.find('span', {'itemprop': 'datePublished'}).text
    except:
        date = ""

    try:
        publisher = bs.find('span', {'itemprop': 'publisher'}).text
    except:
        publisher = ""

    html = urlopen('https://archive.org/stream' + livre[27:] + livre[27:] + '_djvu.txt')
    bs = BeautifulSoup(html.read(), 'html.parser')
    try:
        texte = bs.find('pre').text
    except:
        texte = "TEXTE PAS OCÉRISÉ"

    titreFormate = re.sub(r'[^a-zA-Z0-9]', '', titre)

    nom = date + "_" + titreFormate[0:20] + '.txt'
    writer.writerow([titre, date, publisher, date + "_" +  titreFormate[0:20] + '.txt'])
    f = open(nom, "a", encoding="utf-8")
    f.write(texte)
    f.close()

g.close()

FileNotFoundError: ignored